In [1]:
# conda install -y gensim
# run cell
# conda uninstall -y boto
# conda install -y boto

import glob
import os
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.wrappers import FastText
import pandas as pd
import pymorphy2
import re
from tqdm import tqdm
import _pickle as cPickle
import pandas as pd

In [2]:
# Путь к папке с csv-файлами корпуса
corpora_paths = ['./cd-sco/corpus/training/', './cd-sco/corpus/dev/', './cd-sco/corpus/test-gold/']

In [3]:
with open(os.path.join(corpora_paths[0], 'SEM-2012-SharedTask-CD-SCO-training-09032012.txt'), 'r') as f:
    lines = f.readlines()
    
with open(os.path.join(corpora_paths[1], 'SEM-2012-SharedTask-CD-SCO-dev-09032012.txt'), 'r') as f:
    lines += f.readlines()
    
with open(os.path.join(corpora_paths[2], 'SEM-2012-SharedTask-CD-SCO-test-cardboard-GOLD.txt'), 'r') as f:
    lines += f.readlines()
    
with open(os.path.join(corpora_paths[2], 'SEM-2012-SharedTask-CD-SCO-test-circle-GOLD.txt'), 'r') as f:
    lines += f.readlines()

In [4]:
len(lines)

103752

In [6]:
# Итоговые данные
data = pd.DataFrame(columns=['Text', 'Tags'])
text = ''

# Все метки, встретившиеся в тексте
cues = []

# Элементы negation scope, не равные слову
scope_not_equal = []

for line in tqdm(lines):

    # Переходим к новому предложению
    if line == '\n':
        data = data.append({'Text': np.nan, 'Tags': np.nan}, ignore_index=True)
        continue

    line = line.split('\t')
    word = line[3]
    text += word + ' '
    
    current_cue = ''
    word_without_cue = ''
    is_cue_prefix = False
    is_cue_suffix = False

    if '***' in line[7]:
        negation_tag = 0
    
    else:
        negation_tag = 0

        for i, element in enumerate(line[7:]):

            if i % 3 == 0 and '_' not in element:
                cues.append(element)
            
            if i % 3 == 2 and '_' not in element:
                if element.rstrip() != word:
                    scope_not_equal.append((word, element))
                    
                    if '_' not in line[7 + i - 2]:
                        current_cue = line[7 + i - 2]
                        if word.startswith(current_cue):
                            word_without_cue = word[len(current_cue):]
                            is_cue_prefix = True
                        elif word.endswith(current_cue):
                            word_without_cue = word[:-len(current_cue)]
                            is_cue_suffix = True

                negation_tag = 1
        
    if is_cue_prefix:
        data = data.append({'Text': current_cue, 'Tags': 0}, ignore_index=True)
        data = data.append({'Text': word_without_cue, 'Tags': negation_tag}, ignore_index=True)
    elif is_cue_prefix:
        data = data.append({'Text': word_without_cue, 'Tags': negation_tag}, ignore_index=True)
        data = data.append({'Text': current_cue, 'Tags': 0}, ignore_index=True)
    else:
        data = data.append({'Text': word, 'Tags': negation_tag}, ignore_index=True)


  6%|▋         | 6490/103752 [00:30<07:30, 216.10it/s]
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 103752/103752 [11:40<00:00, 148.21it/s]


In [7]:
len(data)

103915

In [14]:
text[:100]

'Chapter 1. Mr. Sherlock Holmes Mr. Sherlock Holmes , who was usually very late in the mornings , sav'

In [9]:
data.head(40)

,Text,Tags
0,Chapter,0
1,1.,0
2,Mr.,0
3,Sherlock,0
4,Holmes,0
5,NaN,NaN
6,Mr.,0
7,Sherlock,0
8,Holmes,0
9,",",0


In [10]:
set(cues)

{'By',
 'Except',
 'Never',
 'No',
 'None',
 'Nor',
 'Not',
 'Nothing',
 'Nowhere',
 'On',
 'Save',
 'absence',
 'absolutely',
 'all',
 'at',
 'by',
 'contrary',
 'dis',
 'fail',
 'far',
 'for',
 'from',
 'im',
 'in',
 'ir',
 'less',
 'means',
 'more',
 "n't",
 'neglected',
 'neither',
 'never',
 'no',
 'nobody',
 'none',
 'nor',
 'not',
 'nothing',
 'nowhere',
 'prevent',
 'rather',
 'refused',
 'save',
 'than',
 'the',
 'un',
 'without',
 'world'}

In [11]:
len(scope_not_equal)

187

In [12]:
scope_not_equal

[('infrequent', 'frequent'),
 ('unfortunate', 'fortunate'),
 ('unknown', 'known\n'),
 ('unambitious', 'ambitious\n'),
 ('incredulously', 'credulously\n'),
 ('unambitious', 'ambitious\n'),
 ('unambitious', 'ambitious'),
 ('unknown', 'known\n'),
 ('unpractical', 'practical\n'),
 ('unimaginative', 'imaginative\n'),
 ('godless', 'god\n'),
 ('unable', 'able\n'),
 ('unhappy', 'happy\n'),
 ('unhappy', 'happy\n'),
 ('unhappy', 'happy\n'),
 ('infinite', 'finite'),
 ('untimely', 'timely\n'),
 ('unable', 'able\n'),
 ('impenetrable', 'penetrable\n'),
 ('impatient', 'patient\n'),
 ('useless', 'use\n'),
 ('unsafe', 'safe\n'),
 ('immaterial', 'material\n'),
 ('intolerable', 'tolerable\n'),
 ('incredible', 'credible\n'),
 ('lifeless', 'life\n'),
 ('unlikely', 'likely'),
 ('indiscreet', 'discreet\n'),
 ('unfortunately', 'fortunately\n'),
 ('unpleasant', 'pleasant\n'),
 ('unmarried', 'married\n'),
 ('impossible', 'possible\n'),
 ('inexplicable', 'explicable\n'),
 ('unable', 'able\n'),
 ('imprudent', 'pr

In [84]:
data.to_csv('sem-2012.csv', sep='\t', index=False)

In [13]:
with open("./data_sem2012/sem-2012.txt", "w") as text_file:
    text_file.write(text)